<a href="https://colab.research.google.com/github/gowripreetham/SJSU_Pycaret/blob/main/PyCaret_Regression_%2B_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio pandas

In [ ]:
!pip install -U git+https://github.com/pycaret/pycaret.git@master


In [ ]:
import gradio as gr
import pandas as pd
from pycaret.regression import *

In [ ]:
# Load dataset and train PyCaret model (regression)
from pycaret.datasets import get_data
data = get_data('boston')
s = setup(data, target='medv', session_id=42, use_gpu=False)
best = compare_models(n_select=1)
save_model(best, "boston_price_model")

In [ ]:
# Load model back for Gradio use
model = load_model("boston_price_model")

def predict_price(CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT):
    row = pd.DataFrame([[CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT]],
                       columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
    prediction = predict_model(model, data=row)
    price = prediction.loc[0, 'prediction_label']
    return f"🏠 Predicted House Price: ${price*1000:.2f}"

inputs = [
    gr.Slider(0, 90, 0.1, label="Per capita crime rate (CRIM)"),
    gr.Slider(0, 100, 18, label="Residential land zoned (ZN)"),
    gr.Slider(0, 30, 2, label="Non-retail business acres (INDUS)"),
    gr.Radio([0, 1], label="Charles River dummy (CHAS)", value=0),
    gr.Slider(0, 1, 0.5, label="Nitric oxides concentration (NOX)"),
    gr.Slider(3, 9, 6, label="Avg rooms per dwelling (RM)"),
    gr.Slider(1, 100, 60, label="Proportion of old houses (AGE)"),
    gr.Slider(1, 12, 5, label="Distance to employment centers (DIS)"),
    gr.Slider(1, 24, 10, label="Accessibility to highways (RAD)"),
    gr.Slider(100, 800, 400, label="Property tax rate (TAX)"),
    gr.Slider(10, 25, 15, label="Pupil-teacher ratio (PTRATIO)"),
    gr.Slider(0, 400, 300, label="Proportion of blacks (B)"),
    gr.Slider(1, 40, 12, label="Lower status population (LSTAT)")
]


In [ ]:
demo = gr.Interface(
    fn=predict_price,
    inputs=inputs,
    outputs="text",
    title="🏘️ PyCaret Boston Housing Price Predictor",
    description="Interactively predict median house prices using a PyCaret regression model."
)

demo.launch(share=True)